In [1]:
import matplotlib.font_manager

system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
indexes_to_remove = [17, 34, 44, 166, 174, 175, 178, 186, 263, 297, 317]
system_fonts = [font for i, font in enumerate(system_fonts) if i not in indexes_to_remove]

In [2]:
from functions import (
    create_blank_pdf,
    find_optimal_placement,
    can_place_image,
    save_pdf, 
    place_image_in_pdf,
    create_text_image,
    random_augment,
    get_all_canvas_bounding_boxes
)

In [3]:
import random
import numpy as np
from reportlab.lib.pagesizes import letter

width, height = int(letter[0]), int(letter[1])
text_list = ["This is sample text for our PDF", "I like GPT and maroons"]


In [ ]:
cnt = 0
change_page = True
existing_boxes = []
while text_list:
    if change_page:
        pdf_path = f"ocr_{cnt}.pdf"
        canvas_obj, (page_width, page_height) = create_blank_pdf(pdf_path)
        change_page = False
    text = text_list[0]
    for _ in range(15):
        words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
        font_path = random.choice(system_fonts)
        font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
        img, img_info = create_text_image(text, words_per_line=words_per_line, font_path=font_path, font_size=font_size)
        img = random_augment(img)
        x_centroid = random.randint(0, width)
        y_centroid = random.randint(0, height)
        if can_place_image(img, x_centroid, y_centroid, existing_boxes, page_width, page_height):
            new_box = place_image_in_pdf(canvas_obj, img, x_centroid, y_centroid)
            existing_boxes.append(new_box)
            text_list.remove(text)
            break
        else:
            position = find_optimal_placement(img, existing_boxes, page_width, page_height)
            if position:
                x_center, y_center = position
                new_box = place_image_in_pdf(canvas_obj, img, x_center, y_center)
                existing_boxes.append(new_box)
                text_list.remove(text)
                break
            else:
                print("Cannot place image without overlaps")
                save_pdf(canvas_obj)
                cnt += 1
                change_page = True
                existing_boxes = []
save_pdf(canvas_obj)


AttributeError: 'PDFDocument' object has no attribute 'pages'